In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader
import math


In [ ]:
import requests
URL = "http://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip"
response = requests.get(URL)
open("har.zip", "wb").write(response.content)

60999314

In [ ]:
from zipfile import ZipFile
file_name = "har.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
    

In [ ]:
f = open("./UCI HAR Dataset/test/X_test.txt", "r")
array = []
line = f.readline()
index = 0
while line:
    line = line.strip("\n")
    line = line.split()
    array.append([])
    for item in line:
        array[index].append(float(item))
    line = f.readline()
    index += 1
f.close()

xy=np.array(array)
xy=xy.astype(np.float64)
print(xy.shape)
x_test = torch.from_numpy(xy[:, :])
print (x_test)

f = open("./UCI HAR Dataset/test/y_test.txt", "r")
array = []
line = f.readline()
index = 0
while line:
    line = line.strip("\n")
    line = line.split()
    array.append([])
    for item in line:
        array[index].append(float(item))
    line = f.readline()
    index += 1
f.close()

xy=np.array(array)
xy=xy.astype(np.float64)

print(xy.shape)
y_test = torch.from_numpy(xy[:, :])



(2947, 561)
tensor([[ 0.2572, -0.0233, -0.0147,  ..., -0.7200,  0.2768, -0.0580],
        [ 0.2860, -0.0132, -0.1191,  ..., -0.6981,  0.2813, -0.0839],
        [ 0.2755, -0.0261, -0.1182,  ..., -0.7028,  0.2801, -0.0793],
        ...,
        [ 0.3500,  0.0301, -0.1158,  ..., -0.6554,  0.2745,  0.1812],
        [ 0.2376,  0.0185, -0.0965,  ..., -0.6597,  0.2648,  0.1876],
        [ 0.1536, -0.0184, -0.1370,  ..., -0.6601,  0.2639,  0.1881]],
       dtype=torch.float64)
(2947, 1)


In [ ]:
f = open("./UCI HAR Dataset/train/y_train.txt", "r")
array = []
line = f.readline()
index = 0
while line:
    line = line.strip("\n")
    line = line.split()
    array.append([])
    for item in line:
        array[index].append(float(item))
    line = f.readline()
    index += 1
f.close()

xy=np.array(array)
xy=xy.astype(np.float64)
print(xy.shape)
y_train = torch.from_numpy(xy[:, :])


f = open("./UCI HAR Dataset/train/X_train.txt", "r")
array = []
line = f.readline()
index = 0
while line:
    line = line.strip("\n")
    line = line.split()
    array.append([])
    for item in line:
        array[index].append(float(item))
    line = f.readline()
    index += 1
f.close()

xy=np.array(array)
xy=xy.astype(np.float64)
print(xy.shape)
x_train = torch.from_numpy(xy[:, :])

(7352, 1)
(7352, 561)


In [ ]:
class DTC(nn.Module):

    def __init__(self,maxpooling_filter):
        super(DTC, self).__init__()
        

        self.cnv1=nn.Conv1d(1, 1, 10)
        self.fc1=nn.Linear(552-maxpooling_filter+1,552-maxpooling_filter+1)
        self.fc2=nn.Linear(552-maxpooling_filter+1,552-maxpooling_filter+1)
        self.lrel=nn.LeakyReLU()
        self.maxpool=nn.MaxPool1d(maxpooling_filter, stride=1,return_indices=True)
        self.maxunpool=nn.MaxUnpool1d(maxpooling_filter, stride=1)
        self.upsample=nn.Upsample(scale_factor=(552/(553-maxpooling_filter)), mode='bilinear')
        self.deconv1=nn.ConvTranspose1d(1, 1, 10)
    
    def encode(self, x):
        x = self.lrel(self.cnv1(x))
        
        x,self.indices= self.maxpool(x)
        x = self.lrel(self.fc1(x))
        return x,self.indices
    def decode(self,x,indice):
        x = self.lrel(self.fc2(x))
        x= (self.maxunpool(x,indice))#.squeeze(dim=3)
        
        x= self.deconv1(x)
        return x



In [ ]:
from torch.optim.lr_scheduler import MultiStepLR
model = DTC(520)
optimizer = torch.optim.Adam(model.parameters())

loss=nn.MSELoss()

In [ ]:
x_train=x_train.float()
x_test=x_test.float()
y_train=y_train.float()
y_test=y_test.float()
y_train=y_train.view(7352,1, 1)
x_train=x_train.view(7352,1, 561)
x_test=x_test.view(2947,1, 561)
y_test=y_test.view(2947,1, 1)
#cuda0 = torch.device('cuda:0')
#y_test.to(cuda0)
#x_test.to(cuda0)
#y_train.to(cuda0)
#x_train.to(cuda0)

In [ ]:
#x_train=x_train.view(7352,1, 561)
#x_train=x_train.float()
x,y=model.encode(x_train)
print(x.shape)
print(x)
print(x.dim())
#input2 = torch.randn(20, 16, 50)
#print(input2)

#input = torch.arange(1, 41, dtype=torch.float32).view( 2,1, 20)
#x=model.encode(input[1])
#print(x.shape)
#print(x)

#print(input[1].shape)
print(x_train.shape)

torch.Size([7352, 1, 33])
tensor([[[-1.1196e-04,  8.7408e-01,  4.4060e-01,  ...,  4.8807e-01,
          -7.9237e-03,  1.3752e-01]],

        [[-1.1139e-04,  8.7666e-01,  4.4275e-01,  ...,  4.8949e-01,
          -7.9525e-03,  1.3857e-01]],

        [[-1.1188e-04,  8.7444e-01,  4.4089e-01,  ...,  4.8826e-01,
          -7.9277e-03,  1.3767e-01]],

        ...,

        [[-1.5139e-04,  6.9811e-01,  2.9378e-01,  ...,  3.9087e-01,
          -5.9620e-03,  6.6101e-02]],

        [[-1.5219e-04,  6.9452e-01,  2.9080e-01,  ...,  3.8889e-01,
          -5.9221e-03,  6.4647e-02]],

        [[-1.5451e-04,  6.8415e-01,  2.8214e-01,  ...,  3.8316e-01,
          -5.8064e-03,  6.0436e-02]]], grad_fn=<LeakyReluBackward0>)
3
torch.Size([7352, 1, 561])


In [ ]:
class Datatrain(Dataset):
  def __init__(self,train_x,train_y):
    self.x = train_x
    self.y = train_y
  def __len__(self):
    return len(self.x)
  def __getitem__(self,index):
    return self.x[index], self.y[index]

In [ ]:
hartrain = Datatrain(x_train,y_train)
batch_size = 500
data = DataLoader(hartrain,batch_size=batch_size,shuffle=True,num_workers=3)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
scheduler = MultiStepLR(optimizer, milestones=[40,61], gamma=0.1)

In [ ]:
model.train()
for epoch in range(50):
  for features,labels in data:
    optimizer.zero_grad()
    x_pred,indice=model.encode(features)
    
    x_pred=model.decode(x_pred,indice)
    
    l=loss(x_pred,features)
    
    l.backward()
    optimizer.step()
  print(l)


tensor(0.8287, grad_fn=<MseLossBackward0>)
tensor(0.8029, grad_fn=<MseLossBackward0>)
tensor(0.7513, grad_fn=<MseLossBackward0>)
tensor(0.7505, grad_fn=<MseLossBackward0>)
tensor(0.7428, grad_fn=<MseLossBackward0>)
tensor(0.7181, grad_fn=<MseLossBackward0>)
tensor(0.6831, grad_fn=<MseLossBackward0>)
tensor(0.6621, grad_fn=<MseLossBackward0>)
tensor(0.6552, grad_fn=<MseLossBackward0>)
tensor(0.6317, grad_fn=<MseLossBackward0>)
tensor(0.6207, grad_fn=<MseLossBackward0>)
tensor(0.5986, grad_fn=<MseLossBackward0>)
tensor(0.5937, grad_fn=<MseLossBackward0>)
tensor(0.5768, grad_fn=<MseLossBackward0>)
tensor(0.5622, grad_fn=<MseLossBackward0>)
tensor(0.5479, grad_fn=<MseLossBackward0>)
tensor(0.5367, grad_fn=<MseLossBackward0>)
tensor(0.5143, grad_fn=<MseLossBackward0>)
tensor(0.5079, grad_fn=<MseLossBackward0>)
tensor(0.5060, grad_fn=<MseLossBackward0>)
tensor(0.4921, grad_fn=<MseLossBackward0>)
tensor(0.4772, grad_fn=<MseLossBackward0>)
tensor(0.4683, grad_fn=<MseLossBackward0>)
tensor(0.45

In [ ]:
model.eval()
x,y=hartrain[0]
print(x)
x1,x2=model.encode(x)
y=model.decode(x1,x2)
print(y)
print (x1.shape)
print (y.shape)
print (loss(x,y))


tensor([[ 0.2886, -0.0203, -0.1329, -0.9953, -0.9831, -0.9135, -0.9951, -0.9832,
         -0.9235, -0.9347, -0.5674, -0.7444,  0.8529,  0.6858,  0.8143, -0.9655,
         -0.9999, -0.9999, -0.9946, -0.9942, -0.9876, -0.9432, -0.4077, -0.6793,
         -0.6021,  0.9293, -0.8530,  0.3599, -0.0585,  0.2569, -0.2248,  0.2641,
         -0.0952,  0.2789, -0.4651,  0.4919, -0.1909,  0.3763,  0.4351,  0.6608,
          0.9634, -0.1408,  0.1154, -0.9852, -0.9817, -0.8776, -0.9850, -0.9844,
         -0.8947,  0.8921, -0.1613,  0.1247,  0.9774, -0.1232,  0.0565, -0.3754,
          0.8995, -0.9709, -0.9755, -0.9843, -0.9888, -0.9177, -1.0000, -1.0000,
          0.1138, -0.5904,  0.5911, -0.5918,  0.5925, -0.7454,  0.7209, -0.7124,
          0.7113, -0.9951,  0.9957, -0.9957,  0.9917,  0.5702,  0.4390,  0.9869,
          0.0780,  0.0050, -0.0678, -0.9935, -0.9884, -0.9936, -0.9945, -0.9862,
         -0.9928, -0.9852, -0.9920, -0.9931,  0.9898,  0.9920,  0.9905, -0.9935,
         -0.9999, -0.9998, -

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
optimizer2 = torch.optim.SGD(model.parameters(),lr=0.01)
model.train()

DTC(
  (cnv1): Conv1d(1, 1, kernel_size=(10,), stride=(1,))
  (fc1): Linear(in_features=33, out_features=33, bias=True)
  (fc2): Linear(in_features=33, out_features=33, bias=True)
  (lrel): LeakyReLU(negative_slope=0.01)
  (maxpool): MaxPool1d(kernel_size=520, stride=1, padding=0, dilation=1, ceil_mode=False)
  (maxunpool): MaxUnpool1d(kernel_size=(520,), stride=(1,), padding=(0,))
  (upsample): Upsample(scale_factor=16.727272727272727, mode=bilinear)
  (deconv1): ConvTranspose1d(1, 1, kernel_size=(10,), stride=(1,))
)

In [ ]:
n_clusters=6
for epoch in range(50):
  for features,labels in data:
    optimizer2.zero_grad()
    model2 = AgglomerativeClustering(
        n_clusters=n_clusters, linkage="complete"
    )
    with torch.no_grad():
      x_pred,indice=model.encode(features)
      a,b,c=features.shape
      x_pred=x_pred.view(a,33)
      y_predict=model2.fit_predict(x_pred)
      clf = NearestCentroid()
      clf.fit(x_pred, y_predict)
    qij = torch.ones([batch_size, n_clusters], dtype=torch.float64, requires_grad=True)
    pij = torch.ones([batch_size, n_clusters], dtype=torch.float64, requires_grad=True)
    fj = torch.ones([n_clusters,1], dtype=torch.float64, requires_grad=True)
    sigma1=0   
    for i , feature in enumerate(features):
      sigma1=0
      for j in range(n_clusters):
        sigma1+=((1+torch.sqrt(torch.sum((x_pred[i]-clf.centroids_[j])**2)))**-1)
      for j in range(n_clusters):
        qij[i][j] = (((1+torch.sqrt(torch.sum((x_pred[i]-clf.centroids_[j])**2)))**-1)/sigma1) 
    for j in range(n_clusters):
      for i , feature in enumerate(features):
        print (qij.shape)
        fj[j]+=qij[i][j]   
    sigma1=0
    cluster_loss=0
    for i , feature in enumerate(features):
      for j in range(n_clusters):
        cluster_loss+=pij[i][j]*torch.log(pij[i][j]/qij[i][j])
    cluster_loss.backward()
    optimizer2.step()



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: ignored

In [ ]:
a=torch.randn(1, 23)
b=torch.randn(1, 23)
print(a)
print(b)
c=torch.sqrt(torch.sum((a-b)**2))
print (c)

In [ ]:
print(clf.centroids_.shape)
